In [16]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('..')
import stober_synthesis_utils as stober

from time import gmtime, strftime


In [24]:
slack_webhook = "redacted"

In [2]:
jub_ip = "192.168.1.2"


In [3]:
run_message_macro = 'M98 P"0:/macros/test_send.g"'

In [4]:
move = 'G1 X100 Y100'

In [5]:
def get_reply_count():
    r = requests.get(f'http://{jub_ip}/rr_model?key=seqs')
    return r.json()['result']['reply']

In [20]:
def gcode(gcode):
    messages = []
    reply_count = get_reply_count()
    exec = requests.get(f'http://{jub_ip}/rr_gcode?gcode={gcode}')
    while len(messages) < 5:
        new_reply_count = get_reply_count()
        if new_reply_count > reply_count:
            messg = requests.get(f"http://{jub_ip}/rr_reply")
            if messg.content.decode() == 'Crash detected t3\n':
                ping_slack(f'<!channel> Jubilee SiNP crash detected at {strftime("%Y-%m-%d %H:%M:%S", gmtime())}')
                break
            messages.append(messg)
            reply_count = new_reply_count
        time.sleep(0.01)

    return messages

In [19]:
def ping_slack(message):
    r = requests.post(slack_webhook, json = {"text":message})

In [23]:
messages = gcode(run_message_macro)

In [13]:
messages[1].content.decode()

'Crash detected t3\n'

In [73]:
messages = gcode(move)

b'\n'
b'Crash detected t3\n'
b'reverse trigger triggered t4\n'


KeyboardInterrupt: 

In [74]:
messages

NameError: name 'messages' is not defined

In [66]:
r.content

b'ok'

In [17]:
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2024-10-12 01:17:10'